In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#Load the training data
train_data = pd.read_csv('train_data.csv')
X_train = train_data.drop('label', axis=1).values
Y_train = train_data['label'].values

In [3]:
NumClasses = train_data['label'].nunique()

In [4]:
input_shape = X_train.shape[1]

In [5]:
#Load the testing data
test_data = pd.read_csv('test_data.csv')
X_test = test_data.values

In [6]:
#Normalize pixel values to be between 0 and 1
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [7]:
print(X_test.shape)

(10000, 3073)


In [8]:
#Ensure the number of features matches the input shape expected by the model
if X_test.shape[1] != input_shape:
    X_test = X_test.reshape((X_test.shape[0], -1))

In [9]:
lb = LabelBinarizer()
Y_train = lb.fit_transform(Y_train)

In [10]:
#split the trainig data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state=42)

In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(input_shape,), activation="sigmoid"))
model.add(Dense(64, activation= "sigmoid"))
model.add(Dense(NumClasses, activation="softmax"))

In [15]:
sgd = SGD(0.01)
epochs = 10
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])
H = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs = epochs, batch_size=128)

Epoch 1/10
313/313 [==============================] - 8s 20ms/step - loss: 2.0394 - accuracy: 0.2916 - val_loss: 2.0323 - val_accuracy: 0.2866
Epoch 2/10
313/313 [==============================] - 6s 20ms/step - loss: 2.0216 - accuracy: 0.2958 - val_loss: 2.0154 - val_accuracy: 0.2900
Epoch 3/10
313/313 [==============================] - 6s 19ms/step - loss: 2.0056 - accuracy: 0.2973 - val_loss: 2.0003 - val_accuracy: 0.2911
Epoch 4/10
313/313 [==============================] - 6s 19ms/step - loss: 1.9912 - accuracy: 0.3012 - val_loss: 1.9861 - val_accuracy: 0.2984
Epoch 5/10
313/313 [==============================] - 6s 18ms/step - loss: 1.9780 - accuracy: 0.3034 - val_loss: 1.9737 - val_accuracy: 0.3013
Epoch 6/10
313/313 [==============================] - 6s 18ms/step - loss: 1.9661 - accuracy: 0.3076 - val_loss: 1.9626 - val_accuracy: 0.3005
Epoch 7/10
313/313 [==============================] - 6s 20ms/step - loss: 1.9554 - accuracy: 0.3107 - val_loss: 1.9519 - val_accuracy: 0.3049

In [16]:
#Evaluation 
predictions = model.predict(X_val, batch_size = 128)
print("Validation Set Performance")
print(classification_report(Y_val.argmax(axis=1), predictions.argmax(axis=1), target_names=[str(x) for x in lb.classes_]))

79/79 [==============================] - 1s 10ms/step
Validation Set Performance
              precision    recall  f1-score   support

           0       0.31      0.45      0.37       973
           1       0.33      0.25      0.28       979
           2       0.24      0.09      0.13      1030
           3       0.25      0.04      0.08      1023
           4       0.25      0.10      0.15       933
           5       0.26      0.44      0.32      1015
           6       0.27      0.53      0.36       996
           7       0.33      0.20      0.25       994
           8       0.37      0.53      0.44      1017
           9       0.40      0.46      0.43      1040

    accuracy                           0.31     10000
   macro avg       0.30      0.31      0.28     10000
weighted avg       0.30      0.31      0.28     10000



In [ ]:
plt.style.use("ggplot")
plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss")
plt.title("Training and validation Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()